In [51]:
import os
import numpy as np
import os.path as osp
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pathlib import Path
from collections import Counter

from itertools import groupby
from operator import itemgetter

from matplotlib.pyplot import MultipleLocator

In [ ]:
internal_data_pths = [
                      './datasets/LN_classify/Fudan_HN_LN_22-23_RAD/Fudan_HN_LN_230508_patches',
                      './datasets/LN_classify/Fudan_HN_LN_22-23_RAD/Fudan_HN_LN_20231204_patches',
                    # '/nas/xxxxx/datasets/LN_classify/Fudan_HN_LN_22-23_all/Fudan_HN_LN_220528_patches', 
                      ]

external_data_pths = ['./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/CGMH/CGMH_2024_patches',
                      './datasets/LN_classify/Fudan_HN_LN_22-23_RAD/TCGA/TCGA-HNSC_selected_patches',
                      './datasets/LN_classify/Fudan_HN_LN_22-23_RAD/TCGA/TCGA-fixed-RAD_patches',
                      './datasets/LN_classify/Fudan_HN_LN_22-23_RAD/CGMH_Oral/CGMH_Oral_patches',
                      ]

src_root = "./exps/Fudan_HN_LN_paper/final_results"
trg_path = "./Desktop/HN_LN_paper_res_250521_recist"

In [53]:
name_to_sad = {}
for data_idx, crop_pth in enumerate(internal_data_pths + external_data_pths):
    cropfile = osp.join(crop_pth, "cropping_list.csv")
    df = pd.read_csv(cropfile)
    for idx, row in df.iterrows():
        name_to_sad[row['basename'].lower()] = float(row['recist'])

In [54]:
for r in range(5):
    for fd in range(5):
        subdir = "RUN{}/fold{}/RAD_response_add_EENT_TCGA/mobilenetv3_large_25d_fudan_hn_ln_bce_loss_dual_maxpool_ENE".format(r, fd)
        pred_pth = osp.join(src_root, subdir, "predictions.txt")
        newlines = []
        with open(pred_pth, "r") as f:
            lines = f.readlines()
            for l in lines:
                l = l.rstrip()
                casename_w_label = l.split(',')[0].replace('ext_', '').replace('renji_', '').replace('radadded_', '').replace("xm_", '').replace("p2_", "").replace("p2-2_", "").replace("p2-3_", "")
                casename = casename_w_label.replace('_neg', '').replace('_pos', '').replace('_ene', '').lower()
                if casename not in name_to_sad:
                    print("Error: no matched RECIST record for case {}".format(casename))
                    continue
                sad = name_to_sad[casename]
                newlines.append(l + ",{}".format(sad))
        with open(osp.join(trg_path, "RUN{}-fold{}-predictions.txt".format(r, fd)), "w") as f:
            for l in newlines:
                f.write(l + "\n")